NOTE: no error handling implemented

Befeore running install netCDF4 module (https://unidata.github.io/netcdf4-python/netCDF4/index.html) 


Modify the following section settings your local save path and the id of the folder to elaborate from https://github.com/Swiss-Polar-Institute/frictionless-data-packages

In [ ]:
### VARIABLE TO  MODIFY ###
id="10.5281_zenodo.3250980"
basesavepath="c:\\SPI\\"

In [ ]:
from datapackage import Package
from tableschema import Table
import datetime
import os
import urllib.request
from netCDF4 import Dataset
import re

In [ ]:
datapackage_link="https://raw.githubusercontent.com/Swiss-Polar-Institute/frictionless-data-packages/dev/" + id + "/datapackage.json"
tableschema_link="https://raw.githubusercontent.com/Swiss-Polar-Institute/frictionless-data-packages/dev/" + id + "/tableschema.json"
netcdf_name=id +".nc"

In [ ]:
savepath=os.path.join(basesavepath, id)
savepath_datapackage=os.path.join(savepath, "datapackage.json")
savepath_tableschema=os.path.join(savepath, "tableschema.json")
savepath_NetCDF=os.path.join(savepath, netcdf_name)

In [ ]:
os.mkdir(savepath)
urllib.request.urlretrieve(datapackage_link, savepath_datapackage)
urllib.request.urlretrieve(tableschema_link, savepath_tableschema)

In [ ]:
package = Package(savepath_datapackage)
print (package.descriptor['name'])

In [ ]:
dataIsValid=False
for resource in package.resources:
            print(f'Resource: {resource.name} ')
            if resource.tabular is False:               
                continue
            if resource.valid is False:
                print(f'Tableschema: {resource.name} is NOT valid!')
                print(resource.errors)
                continue
            resource.read()
            dataIsValid=True
            print(f'Tableschema: {resource.name} is valid!')
            resourceData=package.get_resource(resource.name)            
            break

In [ ]:
#create netcdf
print("create netcdf")
netcdf = Dataset(savepath_NetCDF, "w", format="NETCDF4")

#create dimension
print("create dimension")
row = netcdf.createDimension("row", None) ##unlimited dimension

#create global attribute
print("create global attribute")
if 'name' in package.descriptor:
    netcdf.name=package.descriptor['name']
if 'id' in package.descriptor:
    netcdf.id=package.descriptor['id']
if 'title' in package.descriptor:
    netcdf.title=package.descriptor['title']
if 'description' in package.descriptor:
    netcdf.description=package.descriptor['description']
if 'homepage' in package.descriptor:
    netcdf.homepage=package.descriptor['homepage']
if 'version' in package.descriptor:
    netcdf.version=package.descriptor['version']
if 'created' in package.descriptor:
    netcdf.created=package.descriptor['created']
if 'x_spi_platform_code' in package.descriptor:
    netcdf.platform_code=package.descriptor['x_spi_platform_code']
if 'x_spi_platform_name' in package.descriptor:
    netcdf.platform_name=package.descriptor['x_spi_platform_name']
if 'x_spi_type' in package.descriptor:
    netcdf.type=package.descriptor['x_spi_type']
if 'x_spi_citation' in package.descriptor:
    netcdf.citation=package.descriptor['x_spi_citation']
if 'keywords' in package.descriptor:
    netcdf.keywords=','.join(package.descriptor['keywords'])
netcdf.cdm_data_type="Other"
netcdf.Conventions="COARDS, CF-1.6, ACDD-1.3"
if 'contributors' in package.descriptor:
    if 'title' in package.descriptor['contributors'][0]:
        netcdf.creator_name = package.descriptor['contributors'][0]['title']
    if 'path' in package.descriptor['contributors'][0]:
        netcdf.creator_url = package.descriptor['contributors'][0]['path']
    if 'organisation' in package.descriptor['contributors'][0]:
        netcdf.institution = package.descriptor['contributors'][0]['organisation']
netcdf.creator_type = "institution"  
if 'homepage' in package.descriptor:
    netcdf.DOI=package.descriptor['homepage']
    netcdf.infoUrl = package.descriptor['homepage']
if 'licenses' in package.descriptor:    
    if 'name' in package.descriptor['licenses'][0]:    
        netcdf.license = package.descriptor['licenses'][0]['name']
if 'description' in package.descriptor:
    netcdf.summary = package.descriptor['description']
if 'contributors' in package.descriptor:
    netcdf.contributors=str(package.descriptor['contributors'])

#create variables
print("create variables")
time = netcdf.createVariable("time","f8",("row",))
for field in resourceData.schema.fields:
    fieldName=field.name.replace(" ","")
    fieldName=re.sub(r'\[.*\]','',fieldName)
    print("fieldName=" + fieldName)
    fieldtype=""
    fieldsize=""
    if field.type=="string":
        fieldtype="S1"        
        maxFieldSize=0
        for row in resourceData.iter(keyed = True):
            if len(row[field.name])>maxFieldSize:
                maxFieldSize=len(row[field.name]) 
        fieldsize=str(maxFieldSize)
    elif field.type=="integer":
        fieldtype="i4"
    elif field.type=="time":
        fieldtype="S1"
        fieldsize="4"
    elif field.type=="datetime":
        fieldtype=""
    elif field.type=="number":
        fieldtype="f8"
    
    print("fieldtype=" + fieldtype)
    
    if fieldtype=="":
        continue
    if fieldsize=="":
        newField = netcdf.createVariable(fieldName,fieldtype,("row",))
    else:
        netcdf.createDimension(fieldName + "_strlen", int(fieldsize))
        newField = netcdf.createVariable(fieldName,fieldtype,("row",fieldName + "_strlen",))

    newField.description=field.descriptor['description']
    if 'unit' in field.descriptor:
        newField.unit=field.descriptor['unit']
    if 'x_spi_cf_standard_name' in field.descriptor:
        newField.standard_name=field.descriptor['x_spi_cf_standard_name']
    if 'x_spi_cf_unit' in field.descriptor:
        newField.unit=field.descriptor['x_spi_cf_unit']
    if 'x_spi_cf_attribute' in field.descriptor:
        newField.attribute=field.descriptor['x_spi_cf_attribute']
    
print("fill values to be implemented")    




In [ ]:
netcdf.close()    